In [9]:
import pandas as pd
import os
from glob import glob
import json
from tqdm import tqdm

In [12]:
src_results = "../results/reddit_results/"
text_data = "../text_data/"

categories = ["guncontrol", 'minority', "politics"]
semesters = [('01/01/2017','01/07/2017'), ('01/07/2017','01/01/2018'),
             ('01/01/2018','01/07/2018'), ('01/07/2018','01/01/2019'), ('01/01/2019','01/07/2019')]


In [13]:
def preprocess_comments(cat, ec_val):
    data_path = f"../data" 
    for topic in tqdm(categories):
        i = 0
        for sem in semesters:
            print("Current iteration:", i)
        # prendiamo il df delle EC
            df_EC = pd.read_csv(os.path.join(src_results, f"{topic}/{ec_val}/{ec_val}_{topic}_{i}.csv"))
        # global community stats + users
            df_users_original = pd.read_csv(os.path.join(src_results, f"{topic}/eva_users_merged_{i}_com_stats.csv"))
            df_users = df_users_original[(df_users_original["size"] >= 20)].copy()
            list_ECs = df_EC.community.tolist()
        #a questo punto, abbiamo solo gli utenti nelle EC
            df_EC_users = df_users[df_users['community'].isin(list_ECs)].copy()
            
            print(df_EC_users.community.unique())
            
            df_EC_users["EC"] = ec_val
            ec_users = df_EC_users[["user_id", "EC", "max_label"]].copy()
            ec_users.to_csv(os.path.join(src_results, f"{topic}/{ec_val}_user_TM_{topic}_{i}.csv"), index = False)
            
            comments = list()
            list_temp_df = []
            period0 = datetime.datetime.strptime(sem[0], "%d/%m/%Y").strftime("%d-%m-%Y")
            period1 = datetime.datetime.strptime(sem[1], "%d/%m/%Y").strftime("%d-%m-%Y")
            # retrieve raw text comments from users' json files
            semester_user_comments = os.path.join(f'../topic_comments/{topic}/{topic}_{period0}_{period1}/')
            json_files = [pos_json for pos_json in os.listdir(semester_user_comments) if pos_json.endswith('.json')]
            for f in json_files:
                f = load_json(os.path.join(semester_user_comments, f))
                df = pd.json_normalize(f["comments"],max_level=1)
                list_temp_df.append(df)
            df_text_users = pd.concat(list_temp_df)

            if topic == "minority" and i == 4: 
                df_text_users.clean_text.fillna(df_text_users['body'], inplace=True)
                del df_text_users["body"]
            elif "body" in df_text_users.columns:
                df_text_users["clean_text"] = df_text_users["body"]
                del df_text_users["body"]
                  
            df_final_users = pd.merge(left=df_EC_users[["community", "user_id"]], right = df_text_users, left_on="user_id", 
                                      right_on = "author" , how = "inner")
        

            # structure of the final df
            # useraname | subreddit in which the comment appears | timestamp | category | text | post (bool) | commento (bool) | stats 
            df_final_users.to_csv(os.path.join(f"../text_data/comments/{topic}/{ec_val}_comments_{topic}_{i}.csv"), index = False)
            
            i+=1
            

In [ ]:
preprocess_comments(categories, "EC")
preprocess_comments(categories, "not_EC")

In [15]:
def preprocess_posts(cat, ec_val):
    data_path = f"../topic_posts/" 
    for topic in tqdm(categories):
        i = 0
        for sem in semesters:
            print("Current iteration:", i)
        # prendiamo il df delle EC per vedere l'id delle community EC
            df_community = pd.read_csv(os.path.join(src_results, f"{topic}/{ec_val}/{ec_val}_{topic}_{i}.csv"))
        #prendiamo il df degli utenti nelle community
            df_stats = pd.read_csv(os.path.join(src_results, f"{topic}/eva_users_merged_{i}_com_stats.csv"))
            df_users = df_stats
            coms_list = df_community.community.tolist()
            df_EC_users = df_users[df_users['community'].isin(coms_list)].copy()
            ec_users = df_EC_users[["user_id", "max_label"]].copy()
            ec_users.to_csv(os.path.join(src_results, f"{topic}/{ec_val}_user_TM_{topic}_{i}.csv"), index = False)
            
            # the text data were previously cleaned
            period0 = datetime.datetime.strptime(sem[0], "%d/%m/%Y").strftime("%d-%m-%Y")
            period1 = datetime.datetime.strptime(sem[1], "%d/%m/%Y").strftime("%d-%m-%Y")
            filename = os.path.join(data_path, f'{topic}/{topic}_{period0}_{period1}.csv')
            df_post = pd.read_csv(filename)
            df_filtered_post = df_post[df_post["author"].isin(df_EC_users["user_id"].tolist())].copy()
            df_final = pd.merge(left=df_EC_users[["community", "user_id"]], right = df_filtered_post, left_on="user_id", 
                    right_on = "author" , how = "inner")

            df_final["EC_val"] = ec_val
            df_final.to_csv(os.path.join(f"../text_data/posts/{topic}/{ec_val}_post_{topic}_{i}.csv"), index = False)
            i+=1
    

In [ ]:
preprocess_posts(categories, "EC")
preprocess_posts(categories, "not_EC")

NameError: name 'preprocess_post_text_pipeline' is not defined